In [1]:
pip install cmake


Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install dlib

  Using cached dlib-19.24.0.tar.gz (3.2 MB)
  Created wheel for dlib: filename=dlib-19.24.0-cp38-cp38-macosx_12_0_x86_64.whl size=3650666 sha256=2ca3f1b4961c27caa776b253fbdd9651f7d1f9407ce35a353b15daa261c11399
  Stored in directory: /Users/mahinrahman/Library/Caches/pip/wheels/4c/d8/2d/a83b10e7bf10cd7d8bef36bf4dcd15b0c9ebf98f990bc984dd
Successfully built dlib
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install face-recognition


  Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install opencv-python


     |████████████████████████████████| 46.4 MB 2.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import numpy as np
import face_recognition as fr
import os
import datetime

In [2]:
path = "images" #Folder name
images = []
classNames = []
mylst = os.listdir(path)

#Appending names into a list
for cls in range(1,len(mylst)):
    if mylst[cls]!=".ipynb_checkpoints":
        curren = cv2.imread(f"{path}/{mylst[cls]}")
        images.append(curren)
        x = mylst[cls].split(".")
        if x[0]!='':
            classNames.append(x[0]) 
print(classNames)

#Function for encoding the images
def find_encodings(lst_images):
    encode_lst = []
    for img in lst_images:
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        l = fr.face_locations(img)
        encoding = fr.face_encodings(img,l)[0]
        encode_lst.append(encoding)
    return encode_lst

encodeListForKnownFaces=find_encodings(images)
print("Encoding Completed")



#Webcam
cam = cv2.VideoCapture(0)
att = [] #Attendence
f1 = open("Attendence.txt","w",buffering = 1)


#Taking frame by frame form the webcam
while True:
    ret,frame = cam.read() #ret = garbage error
    
    #Resizing the image
    imgs = cv2.resize(frame,(0,0),None,0.25,0.25)
    imgs = cv2.cvtColor(imgs,cv2.COLOR_BGR2RGB)
    
    
    location = fr.face_locations(imgs)
    encoding = fr.face_encodings(imgs,location)

    for encode,faceloc in zip(encoding,location):
        matches = fr.compare_faces(encodeListForKnownFaces,encode)

        #FaceDist will return an array comparing the webcam image with given images
        faceDist = fr.face_distance(encodeListForKnownFaces,encode)
        
        
        #Taking the index of minumum distance as it will be my desired image
        matchIndex = np.argmin(faceDist)
        
        
        
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            print(name)
            
            #Appending the found faces once into a list
            if name not in att:
                att.append(name)
                f1.write(f"{name} - {datetime.datetime.now()}\n")
                print(att)
                
            #Foour points for the rectange    
            y1,x2,y2,x1 = faceloc
            y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            
            #Center of the face
            center = ((x1+x2)//2,(y1+y2)//2)
            
            #resolution of the frame
            (h,w) = frame.shape[:2] 
            
            #Center of the frame
            center_frame = ((0+w)//2,(0+h)//2)
            
            #rectangle for the face 
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),4)
            cv2.rectangle(frame,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(frame,f"{name}",(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            
            #line on the face frame
            #cv2.line(frame,(x1,y1),(x2,y2),(255,0,255),2)
            #cv2.line(frame,(x2,y1),(x1,y2),(255,0,255),2)
            
            #rectangle for the center frame
            cv2.rectangle(frame,(center_frame[0]+145,center_frame[1]+145),(center_frame[0]-145,center_frame[1]-145),(0,255,0),4)
    
            #Center of the Face text
            cv2.putText(frame,f"Face : {center}",(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            
            #Axis lines
            cv2.line(frame,(center_frame[0],0),(center_frame[0],h),(144, 238, 144),5)
            cv2.line(frame,(0,center_frame[1]),(w,center_frame[1]),(144, 238, 144),5)
            
            #Center of the frame text
            cv2.putText(frame,f"center : {(center_frame[0],center_frame[1])}",(50,100),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            
            #Navigation Statements
            if center[0]<center_frame[0]:
                if center[1]<center_frame[1]: 
                    cv2.putText(frame,f"Go Left {center_frame[0]-center[0]} pixels and go Down {center_frame[1]-center[1]} pixels",(50,200),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                else:
                    cv2.putText(frame,f"Go left {center_frame[0]-center[0]} pixels and go Up {center[1]-center_frame[1]} pixels",(50,200),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            elif center[0]>center_frame[0]:
                if center[1]<center_frame[1]:
                    cv2.putText(frame,f"Go Right {center[0]-center_frame[0]} pixels and go Down {center_frame[1]-center[1]} pixels",(50,200),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                else:
                    cv2.putText(frame,f"Go right {center[0]-center_frame[0]} pixels and go Up {center[1]-center_frame[1]} pixels",(50,200),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                    
        
                    
            
    cv2.imshow("Webcame",frame)
    key = cv2.waitKey(1)
    if key == 27:
        cam.release()
        cv2.destroyAllWindows()
        break 
f1.close()
    


['Tausif', 'Goni', 'shafin', 'shamma', 'abrar', 'Wasikursaki']
Encoding Completed
WASIKURSAKI
['WASIKURSAKI']
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WASIKURSAKI
WA

KeyboardInterrupt: 

## 